# Flow resistivity measurement

based on ISO 9053-1

The following notebook can be used for measurements of flow rate and pressure sequence, which can be further used for flow resistence and resistivity determination.

**Be careful** about the `target_flow_rate` span. The current setup cannot easily go under **0.5 l/min** and values above **1.5** (for 45 mm specimen diameter) is near the limit given by the standard. In general, it is possible to go higher, it is however necessary to stay in linear region (the line given by pressure to flow rate should not bend for the higher measured values).

## Initial imports and preparations

The following cell imports all the necassary modules and classes and tries to connect to all the devices. Make sure that all the devices are connected and running before running that cell.

In [ ]:
from flowres import FestoFlowActuator, TSI4100FlowMeter, PressureSensor, measurement_loop, export_csv, flow_rate_array, iso_calculation
import os
import logging
logging.basicConfig(level=logging.INFO)
import numpy as np
from time import strftime

# initiation of the sensor
flow_sensor = TSI4100FlowMeter("COM1")
flow_actuator = FestoFlowActuator()
pressure_sensor = PressureSensor()

## Measurement
The following two cells define the specimen, the flow rate span and steps and run the measurment itself

### Specimen definition

In [ ]:
name = "test"
timestamp = strftime("%y-%m-%d_%H-%M")
radius = 0.0225

start_flow = 0.5
end_flow = 1.5
steps = 100
two_way = True
target_flow_rates = flow_rate_array(start_flow, end_flow, steps, two_way)

### Measurement

In [ ]:
try:
    flow_sensor.open()
    pressure_sensor.open()
    
    pressure, flow_rate = measurement_loop(pressure_sensor, flow_sensor, flow_actuator, target_flow_rates)
    data = np.stack((pressure, flow_rate), axis=1)
    export_csv(data, f"{timestamp}_{name}.csv")
finally:
    flow_sensor.close()
    pressure_sensor.close()

## Post-processing and export

The next step is linear regression of the results and figure showing the curve together with the std span.

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import linregress

flow_rate_m3s = flow_rate / 1000 / 60
velocity_mms = flow_rate_m3s / (np.pi*radius**2) * 1000

# linear regression
reg = linregress(pressure, velocity_mms)
pressure_reg = np.array([0, pressure[-1]])
velocity_reg = pressure_reg*reg.slope
# std envelopes
velocitystd_reg_top = pressure_reg*(reg.slope+reg.stderr) + reg.intercept+reg.intercept_stderr
velocitystd_reg_bottom = pressure_reg*(reg.slope-reg.stderr) + reg.intercept-reg.intercept_stderr

# plots
plt.scatter(pressure,velocity_mms, alpha=0.5, label="measured points");
plt.fill_between(pressure_reg, velocitystd_reg_top, velocitystd_reg_bottom, alpha=0.7, label=f"regression +- stderr")
plt.plot(pressure_reg, velocity_reg, label=f"regression\n(offset {reg.intercept:.3f} +- {reg.intercept_stderr:.3f})")
plt.legend()
plt.xlabel("pressure[Pa]")
plt.ylabel("velocity[mm/s]")
plt.savefig(f"{timestamp}_{name}.png")

In [ ]:
diameter = 0.045

radius = diameter/2
surface_area = np.pi*radius**2
results = iso_calculation(flow_rates=flow_rate, pressures=pressure, surface_area=surface_area)
results.export(filename=f"{timestamp}_{name}")